# **LangChain Tools & Agents - Análise Financeira com Function Calling**

**Autor**: Renan Santos Mendes

**Email**: renansantosmendes@gmail.com

**Descrição**: Este notebook demonstra a criação de **agentes inteligentes** usando LangChain que podem acessar e executar ferramentas customizadas para análise de ações financeiras. Exploramos dois padrões principais:

1. **Function Calling direto** - LLM com `bind_tools()` para identificar e sugerir chamadas de ferramentas
2. **Agentes ReAct** - Sistemas autônomos que raciocinam, planejam e executam ferramentas automaticamente

## 🎯 Objetivos de Aprendizado

- Criar ferramentas customizadas com o decorador `@tool`
- Vincular ferramentas a LLMs usando `bind_tools()`
- Entender o padrão ReAct (Reasoning + Acting)
- Construir agentes que tomam decisões sobre qual ferramenta usar
- Monitorar execuções com LangSmith

## 🛠️ Ferramentas Implementadas

- **get_stock_price**: Obtém preço atual e variação percentual
- **get_dividend_yield**: Calcula rendimento de dividendos
- **get_market_cap**: Retorna valor de mercado em bilhões

## 📊 Casos de Uso

Perguntas que o agente pode responder automaticamente:
- "Qual o preço da Apple?"
- "Qual ação tem maior dividend yield: Apple ou Microsoft?"
- "Qual a capitalização de mercado da Tesla?"

## **Instalação dos Pacotes**

In [1]:
%%capture --no-stderr
!pip install langchain langchain-openai langchain-core langchain-community
!pip install langsmith langgraph

## **Imports dos Pacotes Usados no Notebook**

Este conjunto de imports configura um ambiente para criar **agentes de IA conversacionais** usando LangChain e modelos da OpenAI. Inclui ferramentas para
- manipulação de arquivos e sistema operacional (`os`),
- acesso ao Google Drive e dados do Colab (`google.colab`),
- consulta de informações financeiras via Yahoo Finance (`yfinance`),
- toda a infraestrutura do LangChain para construir agentes ReAct que podem raciocinar, usar ferramentas customizadas, manter conversas estruturadas e executar ações baseadas em prompts parametrizáveis, com suporte a anotações de tipo para melhor documentação do código.

In [103]:
# Standard library imports
import os
from typing import Dict, List

# Third-party imports
import yfinance as yf
from google.colab import drive, userdata

# LangChain imports
from langchain.agents import AgentExecutor, create_react_agent
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import AIMessage, HumanMessage
from langchain_core.prompts import PromptTemplate
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI

## **Configuração das Variáveis de Ambiente**

Configuração de variáveis de ambiente para OpenAI e LangSmith

In [3]:
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_KEY')
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGSMITH_KEY')
os.environ["LANGCHAIN_PROJECT"] = "prompt-engineering-techniques"

## **Criação das Tools**

A próxima célula define três ferramentas financeiras que agentes LangChain podem usar para analisar ações.
- (`get_stock_price`) retorna o preço atual e variação percentual de uma ação
- (`get_dividend_yield`) calcula o rendimento de dividendos em porcentagem
- (`get_market_cap`) obtém o valor de mercado da empresa em bilhões de dólares.

Todas utilizam o decorador `@tool` do LangChain para serem automaticamente integradas aos agentes e buscam dados via Yahoo Finance usando o ticker da ação como parâmetro.

In [4]:
@tool
def get_stock_price(ticker: str) -> dict:
    """Get current stock price"""
    stock = yf.Ticker(ticker)
    info = stock.info
    return {
        "ticker": ticker,
        "price": info.get("currentPrice", 0),
        "change_percent": info.get("regularMarketChangePercent", 0)
    }

@tool
def get_dividend_yield(ticker: str) -> float:
    """Get dividend yield percentage"""
    stock = yf.Ticker(ticker)
    return stock.info.get("dividendYield", 0) * 100

@tool
def get_market_cap(ticker: str) -> float:
    """Get market capitalization in billions"""
    stock = yf.Ticker(ticker)
    return stock.info.get("marketCap", 0) / 1e9

In [18]:
tools = [
    get_stock_price,
    get_dividend_yield,
    get_market_cap
]

In [19]:
tools_map = {tool.name: tool for tool in tools}

In [20]:
tools_map

{'get_stock_price': StructuredTool(name='get_stock_price', description='Get current stock price', args_schema=<class 'langchain_core.utils.pydantic.get_stock_price'>, func=<function get_stock_price at 0x7c697bee8b80>),
 'get_dividend_yield': StructuredTool(name='get_dividend_yield', description='Get dividend yield percentage', args_schema=<class 'langchain_core.utils.pydantic.get_dividend_yield'>, func=<function get_dividend_yield at 0x7c6a5f08dd00>),
 'get_market_cap': StructuredTool(name='get_market_cap', description='Get market capitalization in billions', args_schema=<class 'langchain_core.utils.pydantic.get_market_cap'>, func=<function get_market_cap at 0x7c697c0ba7a0>)}

## **Criando um LLM que tem acesso às Tools**

- Inicialização do modelo GPT-4o-mini e utilização do método `bind_tools()` para **vincular as três ferramentas financeiras ao LLM**, permitindo que o modelo identifique automaticamente quando precisa usar essas funções para responder perguntas.
- O `bind_tools()` converte as ferramentas Python em um formato que o modelo entende (function calling), habilitando-o a decidir qual ferramenta chamar baseado na pergunta do usuário.
- Quando invocado com "Qual o preço da Apple?", o modelo reconhece que precisa usar `get_stock_price` e retorna uma resposta estruturada solicitando a execução dessa ferramenta com o ticker apropriado

In [42]:
llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(tools)

response = llm_with_tools.invoke("Qual o preço da ação da Apple e da Microsoft?")

In [43]:
response

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_TIbns8vRxmcXjc0Amp7RuCiz', 'function': {'arguments': '{"ticker": "AAPL"}', 'name': 'get_stock_price'}, 'type': 'function'}, {'id': 'call_5Orf7j9LmpqjIUzGNJUgDF2H', 'function': {'arguments': '{"ticker": "MSFT"}', 'name': 'get_stock_price'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 97, 'total_tokens': 145, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CPZzr8UpLLPWDqNJfLCOJ9FLLILKD', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--e2d23c50-b524-4e75-a202-3213834b8a1e-0', tool_calls=[{'name': 'get_stock_price', 'args': {'ticker': 'AAPL'}, 'id': 'call_TIbns8vRx

In [44]:
response.tool_calls

[{'name': 'get_stock_price',
  'args': {'ticker': 'AAPL'},
  'id': 'call_TIbns8vRxmcXjc0Amp7RuCiz',
  'type': 'tool_call'},
 {'name': 'get_stock_price',
  'args': {'ticker': 'MSFT'},
  'id': 'call_5Orf7j9LmpqjIUzGNJUgDF2H',
  'type': 'tool_call'}]

In [45]:
for tool_call in response.tool_calls:
    print(tool_call)
    print(tool_call.get("args"))
    print(tool_call.get("name"))

    print("Executando a tool manualmente...")

    tool_to_execute = tools_map.get(tool_call.get("name"))
    tool_result = tool_to_execute.invoke(tool_call.get("args"))

    print("Resposta da Tool")
    print(tool_result)


{'name': 'get_stock_price', 'args': {'ticker': 'AAPL'}, 'id': 'call_TIbns8vRxmcXjc0Amp7RuCiz', 'type': 'tool_call'}
{'ticker': 'AAPL'}
get_stock_price
Executando a tool manualmente...
Resposta da Tool
{'ticker': 'AAPL', 'price': 245.27, 'change_percent': -3.45221}
{'name': 'get_stock_price', 'args': {'ticker': 'MSFT'}, 'id': 'call_5Orf7j9LmpqjIUzGNJUgDF2H', 'type': 'tool_call'}
{'ticker': 'MSFT'}
get_stock_price
Executando a tool manualmente...
Resposta da Tool
{'ticker': 'MSFT', 'price': 510.96, 'change_percent': -2.1899}


## **Criação de um Agente que tenha acesso às Tools**

In [130]:
prompt = ChatPromptTemplate.from_messages([
    ("system", """Você é um especialista financeiro experiente com profundo conhecimento em:
    - Análise de ações e mercado financeiro
    - Avaliação de investimentos
    - Interpretação de indicadores financeiros
    - Estratégias de investimento

    Você tem acesso a ferramentas que fornecem dados em tempo real sobre:
    - Preços de ações
    - Dividend yield (rendimento de dividendos)
    - Market cap (capitalização de mercado)

    Ao responder:
    1. Use as ferramentas disponíveis para obter dados atualizados
    2. Forneça análises claras e objetivas
    3. Explique os números em termos que investidores possam entender
    4. Quando relevante, contextualize os dados (ex: comparar com médias do setor)
    5. Seja preciso com os valores e sempre mencione a moeda quando aplicável

    Para ações brasileiras, use o formato correto do ticker (ex: PETR4.SA, VALE3.SA).
    Para ações americanas, use apenas o símbolo (ex: AAPL, MSFT, GOOGL).

    Seja profissional, mas acessível. Seu objetivo é ajudar o usuário a entender melhor seus investimentos.
    """),
    ("human", "{messages}")
])

In [131]:
from langgraph.prebuilt import create_react_agent

model = ChatOpenAI(model="gpt-4o-mini")

agent_executor = create_react_agent(
    model=model,
    tools=tools,
    prompt=prompt)

In [136]:
%%time
result = agent_executor.invoke({
    "messages": "Qual ação tem maior dividend yield: Apple ou Microsoft?",
})

CPU times: user 109 ms, sys: 15.5 ms, total: 125 ms
Wall time: 4.99 s


In [137]:
for message in result.get("messages"):
    print(message)

content='Qual ação tem maior dividend yield: Apple ou Microsoft?' additional_kwargs={} response_metadata={} id='1a53fab6-e2ec-48b3-a1de-b3d76c8d6ade'
content='' additional_kwargs={'tool_calls': [{'id': 'call_bZLkv8sDc9PA6cT9lGb7XUDL', 'function': {'arguments': '{"ticker": "AAPL"}', 'name': 'get_dividend_yield'}, 'type': 'function'}, {'id': 'call_MTES6l4bf8gRkyXB00C2im3k', 'function': {'arguments': '{"ticker": "MSFT"}', 'name': 'get_dividend_yield'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 380, 'total_tokens': 432, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CPbgNw49bk4tLjoMEfMWISCf70zaF', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': N

In [123]:
print(
    {
        "input": 'query',
        "output": result["messages"][-1].content,
    }
)

{'input': 'query', 'output': 'Atualmente, a Apple (AAPL) apresenta um rendimento de dividendos (dividend yield) de **42.0%**, enquanto a Microsoft (MSFT) tem um yield de **71.0%**.\n\nIsso significa que, em termos de rendimento de dividendos, a Microsoft está atualmente oferecendo um retorno maior aos seus acionistas em relação ao que a Apple oferece. O dividend yield é uma métrica importante para investidores que buscam rendimento em ações, especialmente para aqueles focados em estratégias de investimento em dividendos.\n\nSe você tiver mais perguntas sobre essas ações ou sobre investimentos em geral, fique à vontade para perguntar!'}
